In [0]:
gold_patient_visits = spark.sql("""
SELECT
  p.patient_id,
  COUNT(e.encounter_id) AS total_visits,
  MIN(e.start_time) AS first_visit,
  MAX(e.end_time) AS last_visit
FROM sathyajith.demoprojectfhir.silver_patient p
LEFT JOIN sathyajith.demoprojectfhir.silver_encounter e
ON p.patient_id = regexp_replace(e.patient_reference, 'urn:uuid:', '')
GROUP BY p.patient_id
""")


In [0]:
%sql
create table if not exists sathyajith.demoprojectfhir.gold_patient_visits

In [0]:
gold_patient_visits.write \
  .format("delta") \
  .mode("overwrite") \
      .option("mergeSchema", "true")\
  .saveAsTable("sathyajith.demoprojectfhir.gold_patient_visits")


In [0]:
%sql
create table if not exists sathyajith.demoprojectfhir.gold_observation_summary

In [0]:
spark.sql("""
SELECT
  patient_reference,
  observation_name,
  AVG(value) AS avg_value
FROM sathyajith.demoprojectfhir.silver_observation
GROUP BY patient_reference, observation_name
""").write \
  .format("delta") \
  .mode("overwrite") \
      .option("mergeSchema", "true")\
  .saveAsTable("sathyajith.demoprojectfhir.gold_observation_summary")


In [0]:
%sql
select * from sathyajith.demoprojectfhir.gold_patient_visits
    


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
select * from sathyajith.demoprojectfhir.gold_observation_summary

Databricks visualization. Run in Databricks to view.

In [0]:
dedup_patient_df = spark.table(
    "sathyajith.demoprojectfhir.silver_patient"
)


In [0]:
from pyspark.sql.functions import col

gold_patient_df = (
    dedup_patient_df
    .select(
        col("patient_id"),
        col("family_name"),
        col("given_names"),
        col("gender"),
        col("birthDate"),
        col("ingestion_time"),
        col("source_file")
    )
)

gold_patient_df.display()


In [0]:
%sql
create table if not exists sathyajith.demoprojectfhir.gold_dim_patient

In [0]:
gold_patient_df.write \
    .format("delta") \
    .mode("overwrite") \
        .option("mergeSchema", "true")\
    .saveAsTable("sathyajith.demoprojectfhir.gold_dim_patient")


In [0]:
from pyspark.sql.functions import count

gold_patient_df.groupBy("patient_id") \
    .count() \
    .filter(col("count") > 1) \
    .display()


In [0]:
gold_patient_df.filter(
    col("patient_id").isNull() |
    col("gender").isNull()
).display()


In [0]:
gold_patient_df.filter(
    ~col("gender").isin("male", "female", "unknown", "other")
).display()


In [0]:
from pyspark.sql.functions import datediff, current_date

gold_patient_df.withColumn(
    "age",
    datediff(current_date(), col("birthDate")) / 365
).filter(col("age") > 20).display()


In [0]:
spark.read.format("json").load("/Volumes/sathyajith/demoprojectfhir/rawdata/Adela471_Danica886_Schmitt836_0c1a1859-29c7-4f11-f21c-20a7099e8613.json")